In [ ]:
# Import packages
from nba_api.stats.endpoints import shotchartdetail
import json
import requests
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

## Load Data 

In order to make our API request, we need player ID and team ID number to retrieve shot stats. Luckily, we can take advantage of another Github [repo](https://github.com/bttmly/nba). We can use the two JSON files via Python to load them into arrays of dictionary objects. 


In [5]:
# Load teams file
teams = json.loads(requests.get('https://raw.githubusercontent.com/bttmly/nba/master/data/teams.json').text)
# Load players file
players = json.loads(requests.get('https://raw.githubusercontent.com/bttmly/nba/master/data/players.json').text)

In [6]:
# Get team ID based on team name
def get_team_id(team):
  for team in teams:
    if team['teamName'] == team:
      return team['teamId']
  return -1
# Get player ID based on player name
def get_player_id(first, last):
  for player in players:
    if player['firstName'] == first and player['lastName'] == last:
      return player['playerId']
  return -1

In [8]:
# Get the team ID number for the Knicks
get_team_id('Golden State Warriors')


# Get the player ID number for Klay Thompson
get_player_id('Klay', 'Thompson')


202691